In [ ]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="1" # NVIDIA GeForce RTX 3090
os.environ["CUDA_VISIBLE_DEVICES"]="3" # NVIDIA GeForce RTX 2080
 
import tensorflow.compat.v2 as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
gpus = tf.config.experimental.list_physical_devices('GPU')
 
print(gpus)

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
import zipfile
import os
import random
import datetime
import h5py
import cv2
import sklearn.metrics 
from functions import *

In [ ]:
# Name of model
name_model = "Final (M0_5(3x3)_2)"
#version = ".3"

# Path to directory containing the model
filepath_dic = "Results/" + name_model
#filepath_subdic = filepath_dic + "/" + name_model + version

# Path to model
filepath_checkpoint_model = filepath_dic + "/checkpoint.model.keras"

# Path for saving the prediction
name_val_prediction = filepath_dic + "/val_prediction.pkl"

# Prediction

## Validation set

In [ ]:
# Load validation set
fileHDFVal = 'Data/val.hdf5'
val = h5py.File(fileHDFVal, 'r')

# Images of validation set
val_features = np.asarray(val["features"])
val_targets = np.asarray(val["targets"])

## Prediction

In [ ]:
# Settings
model_best = tf.keras.models.load_model(filepath_checkpoint_model)
batch_size = 1

In [ ]:
# Prediction
val_predictions = model_best.predict(
    val_features,
    batch_size=batch_size)

In [ ]:
# Save predictions
val_predictions = val_predictions.reshape(val_targets.shape)

with open(name_val_prediction, 'wb') as file:
    pickle.dump(val_predictions, file)

## Evaluation

In [ ]:
with open("Data/val_coordinates.pkl", 'rb') as file:
    val_coordinates = pickle.load(file)

bad = 26,40
plt.xlim([1200,1400])
plt.ylim([250,450])
plt.xlim([800,1000])
plt.ylim([1000,1200])

### Visual evaluation

In [ ]:
#i = random.choice(range(len(val_predictions)))
i = 26
plots = [val["features"][i], val["targets"][i], val_predictions[i]]

plt.figure(figsize=(24,8))
for j in range(3):
    plt.subplot(1,3,j+1)
    plt.imshow(plots[j], interpolation='none')
    plt.axis("off")
    if j == 0:
        plt.scatter(*zip(*val_coordinates[i]),s=5, c="orange")

In [ ]:
plt.imshow(val["features"][i], cmap="gray")
plt.scatter(*zip(*val_coordinates[i]),s=10, c="orange")
plt.axis("off")
plt.title("Annotated image")

In [ ]:
plt.imshow(val["targets"][i], interpolation='none')
plt.axis("off")
plt.title("Target")

In [ ]:
plt.imshow(val_predictions[i], interpolation='none')
plt.title("Prediction")
plt.axis("off")
plt.colorbar()

In [ ]:
plots = [val["features"][i], val["targets"][i], val_predictions[i]]

plt.figure(figsize=(24,8))
for j in range(3):
    plt.subplot(1,3,j+1)
    plt.imshow(plots[j], interpolation='none')
    plt.xlim([800,1000])
    plt.ylim([1000,1200])

In [ ]:
plt.figure()
plt.imshow(val["features"][i], cmap="gray")
plt.xlim([800,1000])
plt.ylim([1000,1200])
plt.title("Original image")

In [ ]:
plt.figure()
plt.imshow(val["targets"][i], interpolation='none')
plt.xlim([800,1000])
plt.ylim([1000,1200])
plt.title("Target")
plt.colorbar()

In [ ]:
plt.figure()
plt.imshow(val_predictions[i], interpolation='none')
plt.xlim([800,1000])
plt.ylim([1000,1200])
plt.title("Prediction")
plt.colorbar()

### Evaluation based on metrics

In [ ]:
for i in range(len(val_predictions)):
    val_predictions[i] = normalization(val_predictions[i])

In [ ]:
threshold = 0.1

val_targets_binary = binary(arr=val_targets, threshold=threshold)
val_predictions_binary = binary(arr=val_predictions, threshold=threshold)

In [ ]:
prec = precision(binary_targets=val_targets_binary, binary_predictions=val_predictions_binary)
rec = recall(binary_targets=val_targets_binary, binary_predictions=val_predictions_binary)
f1 = F1(binary_targets=val_targets_binary, binary_predictions=val_predictions_binary)
uoi = jaccard(binary_targets=val_targets_binary, binary_predictions=val_predictions_binary)

In [ ]:
print("{:20s}{:^10s}{:^10s}{:^10s}{:^10s}".format("", "MEAN", "MEDIAN","MIN","MAX"))
print("{:20s}{:^10.2f}{:^10.2f}{:^10.2f}{:^10.2f}".format("precision",np.mean(prec),np.median(prec), np.min(prec),np.max(prec)))
print("{:20s}{:^10.2f}{:^10.2f}{:^10.2f}{:^10.2f}".format("recall",np.mean(rec), np.median(rec),np.min(rec),np.max(rec)))
print("{:20s}{:^10.2f}{:^10.2f}{:^10.2f}{:^10.2f}".format("F1-score",np.mean(f1),np.median(f1),np.min(f1),np.max(f1)))
print("{:20s}{:^10.2f}{:^10.2f}{:^10.2f}{:^10.2f}".format("Jaccard",np.mean(uoi),np.median(uoi), np.min(uoi),np.max(uoi)))